# Other State-of-the-Art Implementations:

The following modules will test the other state of the art implementations in the article in the order of their appearance for 4096 pairs of nucleotide sequences:

### SSW:

In [1]:
! time /ssw/src/ssw_test /others/data/seqx.fasta /others/data/seqy.fasta >ssw_out.txt

CPU time: 0.280679 seconds
0.27user 0.00system 0:00.28elapsed 99%CPU (0avgtext+0avgdata 1044maxresident)k
0inputs+1112outputs (0major+289minor)pagefaults 0swaps


### Parasail:

In [2]:
! time /root/parasail/build/parasail_aligner -f /others/data/seqx.fasta -q /others/data/seqy.fasta >parasail_out.txt

10.36user 0.12system 0:10.62elapsed 98%CPU (0avgtext+0avgdata 202476maxresident)k
0inputs+312outputs (0major+67513minor)pagefaults 0swaps


### SeqAn:

In [3]:
! time /root/seqan/smith_waterman_seqan /vectron/docker/experiments_docker/data/cpp_small/seqy.txt /vectron/docker/experiments_docker/data/cpp_small/seqx.txt >seqan_out.txt

13.46user 0.03system 0:13.53elapsed 99%CPU (0avgtext+0avgdata 11164maxresident)k
0inputs+728outputs (0major+5979minor)pagefaults 0swaps


### Codon (seq) inter_align:

In [4]:
! codon build -plugin /codon-seq/ seq_interalign.codon -release

In [5]:
! time ./seq_interalign >seq_interalign.txt

total took 0.363701s
0.41user 1.48system 0:01.98elapsed 96%CPU (0avgtext+0avgdata 147152maxresident)k
0inputs+48outputs (0major+9852minor)pagefaults 0swaps


### ADEPT: 

ADEPT's test are unable to perform for arbitrary number of pairs with arbitrary lengths, and their DNA test suite of 3000 sequence pairs of length 150 was used to perform benchmarks, **the resulting runtime must be multiplied by 4.66** to get the equivalent of other runtime benchmarks.

In [6]:
! time /root/ADEPT/build/examples/simple_sw/simple_sw /root/ADEPT/test-data/dna-query.fasta /root/ADEPT/test-data/dna-reference.fasta out.txt res.txt


-----------------------
       SIMPLE DNA      
-----------------------

STATUS: Launching driver


STATUS: Writing results...
correctness test passed
0.04user 1.49system 0:01.80elapsed 85%CPU (0avgtext+0avgdata 217172maxresident)k
0inputs+272outputs (0major+28357minor)pagefaults 0swaps


### SWIPE:

In [7]:
! time /root/swipe/swipe -d /swipe/swipe_data/seqx.fasta -i /swipe/swipe_data/seqy.fasta >swipe_out.txt

9.83user 0.07system 0:09.94elapsed 99%CPU (0avgtext+0avgdata 1576maxresident)k
0inputs+19280outputs (0major+14435minor)pagefaults 0swaps


### Compiling RV scripts (equivalent to out C++ scripts with the loop annotation specific to RV)

In [8]:
! clang++-16 -fplugin=/root/libRV.so -mllvm -rv-loopvec -fno-unroll-loops /RV/cpp_sw_rv.cpp -o /RV/cpp_sw_rv
! clang++-16 -fplugin=/root/libRV.so -mllvm -rv-loopvec -fno-unroll-loops /RV/cpp_nmw_rv.cpp -o /RV/cpp_nmw_rv
! clang++-16 -fplugin=/root/libRV.so -mllvm -rv-loopvec -fno-unroll-loops /RV/cpp_mt_rv.cpp -o /RV/cpp_mt_rv
! clang++-16 -fplugin=/root/libRV.so -mllvm -rv-loopvec -fno-unroll-loops /RV/cpp_mcp_rv.cpp -o /RV/cpp_mcp_rv
! clang++-16 -fplugin=/root/libRV.so -mllvm -rv-loopvec -fno-unroll-loops /RV/cpp_lcs_rv.cpp -o /RV/cpp_lcs_rv
! clang++-16 -fplugin=/root/libRV.so -mllvm -rv-loopvec -fno-unroll-loops /RV/cpp_LD_rv.cpp -o /RV/cpp_LD_rv
! clang++-16 -fplugin=/root/libRV.so -mllvm -rv-loopvec -fno-unroll-loops /RV/cpp_hd_rv.cpp -o /RV/cpp_hd_rv

Below prints the runtime of the C++ files using the RV vectorizer:

In [9]:
! time /RV/cpp_sw_rv /vectron/docker/experiments_docker/data/cpp_small/seqy.txt /vectron/docker/experiments_docker/data/cpp_small/seqx.txt >rv_out.txt

50.10user 13.02system 1:03.28elapsed 99%CPU (0avgtext+0avgdata 12866008maxresident)k
0inputs+48outputs (0major+11141059minor)pagefaults 0swaps


In [10]:
! time /RV/cpp_nmw_rv /vectron/docker/experiments_docker/data/cpp_small/seqy.txt /vectron/docker/experiments_docker/data/cpp_small/seqx.txt >rv_out.txt

56.70user 9.57system 1:06.43elapsed 99%CPU (0avgtext+0avgdata 6561736maxresident)k
0inputs+56outputs (0major+4896013minor)pagefaults 0swaps


In [11]:
! time /RV/cpp_mt_rv /vectron/docker/experiments_docker/data/cpp_small/seqy.txt /vectron/docker/experiments_docker/data/cpp_small/seqx.txt >rv_out.txt

42.57user 1.53system 0:44.21elapsed 99%CPU (0avgtext+0avgdata 2194824maxresident)k
0inputs+48outputs (0major+1479417minor)pagefaults 0swaps


In [12]:
! time /RV/cpp_mcp_rv /vectron/docker/experiments_docker/data/cpp_small/seqy.txt /vectron/docker/experiments_docker/data/cpp_small/seqx.txt >rv_out.txt

40.72user 1.50system 0:42.32elapsed 99%CPU (0avgtext+0avgdata 2194820maxresident)k
0inputs+48outputs (0major+1543598minor)pagefaults 0swaps


In [13]:
! time /RV/cpp_lcs_rv /vectron/docker/experiments_docker/data/cpp_small/seqy.txt /vectron/docker/experiments_docker/data/cpp_small/seqx.txt >rv_out.txt

44.50user 1.47system 0:46.08elapsed 99%CPU (0avgtext+0avgdata 2190600maxresident)k
0inputs+40outputs (0major+1476655minor)pagefaults 0swaps


In [14]:
! time /RV/cpp_LD_rv /vectron/docker/experiments_docker/data/cpp_small/seqy.txt /vectron/docker/experiments_docker/data/cpp_small/seqx.txt >rv_out.txt

43.52user 1.45system 0:45.08elapsed 99%CPU (0avgtext+0avgdata 2190596maxresident)k
0inputs+48outputs (0major+1477019minor)pagefaults 0swaps


In [15]:
! time /RV/cpp_hd_rv /vectron/docker/experiments_docker/data/cpp_small/seqy.txt /vectron/docker/experiments_docker/data/cpp_small/seqx.txt >rv_out.txt

26.44user 1.59system 0:28.10elapsed 99%CPU (0avgtext+0avgdata 2194824maxresident)k
0inputs+40outputs (0major+1487671minor)pagefaults 0swaps


### Sample Polyhedral Optimization:

In [16]:
! clang++ -mllvm --polly /vectron/docker/experiments_docker/source/cpp/smith_waterman.cpp -o /vectron/docker/experiments_docker/source/cpp/smith_waterman

In [17]:
! time /vectron/docker/experiments_docker/source/cpp/smith_waterman /vectron/docker/experiments_docker/data/cpp_small/seqy.txt /vectron/docker/experiments_docker/data/cpp_small/seqx.txt >poly_out.txt

51.73user 17.92system 1:09.83elapsed 99%CPU (0avgtext+0avgdata 12865892maxresident)k
0inputs+48outputs (0major+11715087minor)pagefaults 0swaps


## Others

Unfortunately, we were unable to replicate 2 of the software packages we used in our benchmarks on the Docker image: SW#, and Aalign.

#### 1. SW#
SW# utilizes CUDA 11.0 and is not compatible with CUDA 12.0 and later. However, on a Linux or Mac-based system with CUDA 11.0, SW# can be built as follows:

1. Clone the repo:
    ```bash
    git clone https://github.com/mkorpar/swsharp
    ```

2. Go to the project directory and remove the existing Makefile:
    ```bash
    cd swsharp/swsharp
    rm -rf Makefile
    ```

3. Create a new Makefile using either nano or vim and copy and paste the following script:

    ```makefile
    CC = gcc
    CP = g++
    CU = nvcc
    LD = nvcc
    DX = doxygen

    NAME = swsharp

    OBJ_DIR = obj
    SRC_DIR = src
    DOC_DIR = doc
    INC_DIR = ../include/$(NAME)
    LIB_DIR = ../lib
    EXC_DIR = ../bin
    WIN_DIR = ../swsharpwin/$(NAME)

    I_CMD = $(addprefix -I, $(SRC_DIR) )
    L_CMD = $(addprefix -L, )

    DEP_LIBS =

    CC_FLAGS = $(I_CMD) -O3 -Wall -march=native
    CP_FLAGS = $(CC_FLAGS)
    LD_FLAGS = $(I_CMD) $(L_CMD) -lpthread -lstdc++

    CU_FLAGS = $(I_CMD) -O3

    API = $(addprefix $(SRC_DIR)/, align.h alignment.h chain.h constants.h \
            cpu_module.h cuda_utils.h database.h db_alignment.h evalue.h gpu_module.h \
            post_proc.h pre_proc.h reconstruct.h scorer.h swsharp.h thread.h threadpool.h)

    SRC = $(shell find $(SRC_DIR) -type f \( -iname \*.cpp -o -iname \*.c -o -iname \*.cu \))
    HDR = $(shell find $(SRC_DIR) -type f \( -iname \*.h \))
    OBJ = $(subst $(SRC_DIR), $(OBJ_DIR), $(addsuffix .o, $(basename $(SRC))))
    DEP = $(OBJ:.o=.d)
    INC = $(subst $(SRC_DIR), $(INC_DIR), $(API))
    LIB = $(LIB_DIR)/lib$(NAME).a
    EXC = $(NAME)
    BIN = $(EXC_DIR)/$(EXC)
    DOC = $(DOC_DIR)/Doxyfile
    WIN = $(subst $(SRC_DIR), $(WIN_DIR), $(HDR) $(SRC))

    debug: CC_FLAGS := $(CC_FLAGS) -DDEBUG -DTIMERS
    debug: CP_FLAGS := $(CP_FLAGS) -DDEBUG -DTIMERS
    debug: CU_FLAGS := $(CU_FLAGS) -DDEBUG -DTIMERS --ptxas-options=-v

    cpu: LD = $(CC)

    all: $(OBJ) $(DEP_LIBS)
    debug: all
    cpu: all

    install: lib include win

    bin: $(BIN)

    include: $(INC)

    lib: $(LIB)

    win: $(WIN)

    $(EXC): $(OBJ) $(DEP_LIBS)
            @echo [LD] $@
            @mkdir -p $(dir $@)
            @$(LD) $(OBJ) -o $@ $(LD_FLAGS)
    $(OBJ_DIR)/%.o: $(SRC_DIR)/%.c
            @echo [CC] $<
            @mkdir -p $(dir $@)
            @$(CC) $< -c -o $@ -MMD $(CC_FLAGS)
    $(OBJ_DIR)/%.o: $(SRC_DIR)/%.cpp
            @echo [CP] $<
            @mkdir -p $(dir $@)
            @$(CP) $< -c -o $@ -MMD $(CP_FLAGS)

    $(OBJ_DIR)/%.o: $(SRC_DIR)/%.cu
            @mkdir -p $(dir $@)
    ifeq (,$(findstring cpu,$(MAKECMDGOALS)))
            @echo [CU] $<
            @$(CU) $< -M -o $(@:.o=.d) $(CU_FLAGS) --output-directory $(dir $@)
            @$(CU) $< -c -o $@ $(CU_FLAGS)
    else
            @echo [CP] $<
            @$(CP) -x c++ $< -c -o $@ -MMD $(CP_FLAGS)
    endif

    $(INC_DIR)/%.h: $(SRC_DIR)/%.h
            @echo [CP] $@
            @mkdir -p $(dir $@)
            @cp $< $@
            
    $(LIB): $(OBJ)
            @echo [AR] $@
            @mkdir -p $(dir $@)
            @ar rcs $(LIB) $(OBJ) 2> /dev/null

    $(BIN): $(EXC)
            @echo [CP] $@
            @mkdir -p $(dir $@)
            @cp $< $@

    $(WIN_DIR)/%: $(SRC_DIR)/%
            @echo [CP] $@
            @mkdir -p $(dir $@)
            @cp $< $@

    docs:
            @echo [DX] generating documentation
            @$(DX) $(DOC)
            
    clean:
            @echo [RM] cleaning
            @rm -rf $(OBJ_DIR) $(EXC)

    remove:
            @echo [RM] removing
            @rm -rf $(INC_DIR) $(LIB) $(BIN) $(EXC) $(WIN)

    -include $(DEP)
    ```

4. Navigate back to the main swsharp folder and run:
    ```bash
    make
    ```

5. The executables will be in `./swsharp/bin`.

6. A simple alignment test can be run by using:
    ```bash
    ./swsharp/bin/swsharpdb -i query.fasta -j target.fasta
    ```

#### 2. Aalign
Aalign utilizes `icpc` (Intel's C++ compiler), which is not available on Ubuntu 22.04's Docker image. However, on a Linux or Mac-based system with CUDA 11.0, Aalign can be built as follows:

1. Install the Intel `icpc` C++ compiler.
2. Clone the repo:
    ```bash
    git clone https://github.com/vtsynergy/aalign
    ```

3. In `./aalign/ModularDesign`, run:
    ```bash
    make
    ```

4. You will have access to Smith-Waterman (different modes) and Needleman-Wunsch (different modes).
